***
### DISCLAIMER ###

El presente ipython notebook ha sido creado para el curso **ILI286 - Computación Científica 2**, del [Departamento de Informática](http://www.inf.utfsm.cl/), [Universidad Técnica Federico Santa María](http://www.utfsm.cl/). 

El material ha sido creado por Claudio Torres <ctorres@inf.utfsm.cl> y Sebastian Flores <sebastian.flores@usm.cl>, y es distribuido sin restricciones. En caso de encontrar un error, por favor no dude en contactarnos.

[Update 2015] Se ha actualizado los notebooks a Python 3 e includio el "magic" "%matplotlib inline" antes de cargar matplotlib para que los gráficos se generen en el notebook. 
***

## Algunos códigos utilizados en este notebook han sido obtenidos del libro "Lloyd N. Trefethen, Spectral Methods in MATLAB, SIAM, Philadelphia, 2000" y traducidos a python.

# Regiones de Convergencia, EDOs numéricas #

In [ ]:
%matplotlib inline
import matplotlib as plt
from matplotlib import pyplot
import numpy as np
L = 6

def zplot(z, ax=pyplot.gca(), lw=1.5):
  ax.plot(np.real(z), np.imag(z), 'k', lw=lw)

## Adams - Bashforth ##

In [ ]:
def adams_bashforth(ax):
  z = np.exp(1j * np.pi * np.arange(201)/100.)
  r = z-1
  s1 = 1
  s2 = (3.-1./z)/2.
  s3 = (23.-16./z+5./z**2)/12.
  zplot(r/s1, ax)
  zplot(r/s2, ax)
  zplot(r/s3, ax)
  ax.axis("equal")
  ax.axis([-2.5, 0.5, -1.5, 1.5])
  ax.grid("on")
  ax.set_title("Adams-Bashforth")
  return ax

fig = pyplot.figure(figsize=(L,L))
adams_bashforth(fig.gca())

## Adams - Moulton ##

In [ ]:
def adams_moulton(ax):
  z = np.exp(1j * np.pi * np.arange(201)/100.)
  r = z-1
  d = 1-1./z;
  s3 = (5*z+8-1./z)/12.
  s4 = (9*z+19.-5./z+1./z**2)/24.
  s5 = (251*z+646-264./z+106./z**2-19./z**3)/720.
  s6 = 1-d/2-d**2/12-d**3/24-19*d**4/720-3*d**5/160.
  zplot(r/s3, ax)
  zplot(r/s4, ax)
  zplot(r/s5, ax)
  zplot(d/s6, ax)
  ax.axis("equal")
  ax.axis([-7, 1, -4, 4])
  ax.grid("on")
  ax.set_title("Adams-Moulton")
  return ax

fig = pyplot.figure(figsize=(L,L))
adams_moulton(fig.gca())


## Backward Differentiation ##

In [ ]:
# Subplot 3 : Backward differentiation:
def backward_differentiation(ax):
  z = np.exp(1j * np.pi * np.arange(201)/100.)
  r = z-1
  d = 1-1./z;
  r = 0
  for i in np.arange(1.,7.):
    r = r+np.power(d,i)/i
    zplot(r, ax)
  ax.axis("equal")
  ax.axis([-15, 35, -25, 25])
  ax.grid("on")
  ax.set_title('Backward Differentiation')
  return ax

fig = pyplot.figure(figsize=(L,L))
backward_differentiation(fig.gca())


## Runge Kutta ##

In [ ]:
def runge_kutta(ax):
  z = np.exp(1j * np.pi * np.arange(201)/100.)
  r = z-1
  d = 1-1./z;
  # Order 1
  W1, W2, W3, W4 = [0], [0], [0], [0]
  for zi in z[1:]:
    W1.append( W1[-1]-(1.+W1[-1]-zi) )
  for zi in z[1:]:
    W2.append( W2[-1]-(1+W2[-1]+.5*W2[-1]**2-zi**2)/(1+W2[-1]) )
  for zi in z[1:]:
    num = (1+W3[-1]+.5*W3[-1]**2+W3[-1]**3/6.-zi**3)
    den = (1+W3[-1]+W3[-1]**2/2.)
    W3.append( W3[-1] - num/den )
  for zi in z[1:]:
    num = (1+W4[-1]+.5*W4[-1]**2+W4[-1]**3/6+W4[-1]**4/24-zi**4)
    den = (1+W4[-1]+W4[-1]**2/2+W4[-1]**3/6.)
    W4.append( W4[-1] - num/den )
  zplot(W1, ax)
  zplot(W2, ax)
  zplot(W3, ax)
  zplot(W4, ax)
  ax.axis("equal")
  ax.axis([-5, 2, -3.5, 3.5])
  ax.grid("on")
  ax.set_title("Runge-Kutta")
  return ax

fig = pyplot.figure(figsize=(L,L))
runge_kutta(fig.gca())

## Comparación de todos los métodos ##

In [ ]:
fig, axs = pyplot.subplots(2, 2, figsize=(L,L))
adams_bashforth(axs[0,0])
adams_moulton(axs[0,1])
backward_differentiation(axs[1,0])
runge_kutta(axs[1,1])
fig.subplots_adjust(hspace=0.3, wspace=0.2)
pyplot.show()

# Un ejemplo práctico

Se presenta un ejemplo simple de un IVP

In [ ]:
from scipy.integrate import odeint
from pylab import *

def my_f(t,y):
    return np.array(y*(1-y))

def euler_ode(y,t,f,h):
    return y+h*f(t,y)

def RK2_ode(y,t,f,h):
    k1=y+h/2.0*f(t,y) #or euler_ode(y,t,f,h)
    return y+h*f(t+h/2.0,k1)

def RK4_ode(y,t,f,h):
    k1=f(t,y)
    k2=f(t+h/2.0,y+(h/2.0)*k1)
    k3=f(t+h/2.0,y+(h/2.0)*k2)
    k4=f(t+h,y+h*k3)
    return y+(h/6.0)*(k1+2*k2+2*k3+k4)


In [ ]:
y0=-0.2

h=0.1

t_times = np.arange(0, 4, h)
y_output = np.zeros(t_times.size)
y_output[0] = y0
for i in range(1,t_times.size):
    #y_output[i]=euler_ode(y_output[i-1],t_times[i-1],my_f,h)
    #y_output[i]=RK2_ode(y_output[i-1],t_times[i-1],my_f,h)
    y_output[i]=RK4_ode(y_output[i-1],t_times[i-1],my_f,h)
    
fig = pyplot.figure(figsize=(5,5))
ax = fig.gca()
ax.axis("equal")
ax.grid(True)
ax.set_title("Numerical Approximation")
ax.axis([0, 4, -2, 2])

plot(t_times,y_output,'-')
plot(t_times,y_output,'.k')


Ahora con muchas condiciones iniciales

In [ ]:
h=0.1
t_times = np.arange(0, 4, h)
fig = pyplot.figure(figsize=(5,5))
fig.clf()
ax = fig.gca()
ax.axis("equal")
ax.grid(True)
ax.set_title("Numerical Approximation")
ax.axis([0, 4, -2, 2])

for y0 in np.arange(-1,5,0.1): 
    y_output = np.zeros(t_times.size)
    y_output[0] = y0
    for i in range(1,t_times.size):
        #y_output[i]=euler_ode(y_output[i-1],t_times[i-1],my_f,h)
        #y_output[i]=RK2_ode(y_output[i-1],t_times[i-1],my_f,h)
        y_output[i]=RK4_ode(y_output[i-1],t_times[i-1],my_f,h)
    plot(t_times,y_output,'-')

## What about a second order problem?

$$\ddot{y}-\mu\,(1-y^2)\,\dot{y}+y=0$$ with $y(0)=2$ and $\dot{y}(0)=0$.

In [ ]:
def my_vdp(t,y):
    mu=10
    y1=y[0]
    y2=y[1]
    return np.array([y2, mu*(1-y1**2)*y2-y1])

In [ ]:
y0=[2, 0]

h=0.0001

T=40

t_times = np.arange(0, T, h)
y_output = np.zeros([t_times.size,2])
y_output[0,:] = y0
for i in range(1,t_times.size):
    y_output[i,:]=euler_ode(y_output[i-1,:],t_times[i-1],my_vdp,h)
    
fig = pyplot.figure(figsize=(5,5))
ax = fig.gca()
#ax.axis("equal")
plt.xlabel("t")
plt.ylabel("y(t)")
ax.grid(True)
ax.set_title("Numerical Approximation")
#ax.axis([0, 4, -2, 2])

plot(t_times,y_output[:,0],'-')
#plot(t_times,y_output[:,1],'r-')
#plot(t_times,y_output[:,0],'.k')

In [ ]:
y_output[0,:]=RK4_ode(y_output[0,:],t_times[0],my_vdp,h)
print(y_output[0,:])

Now looking at the phase portrait

In [ ]:
fig = pyplot.figure(figsize=(5,5))
ax = fig.gca()
#ax.axis("equal")
ax.grid(True)
ax.set_title("Phase Portrait")
plt.xlabel("y1(t)")
plt.ylabel("y2(t)")
plot(y_output[:,0],y_output[:,1],'-')